#### Package installations
Uncomment if use GPU rent services

In [1]:
# !pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
# !pip install nuscenes-devkit

In [3]:
# # Fix error : libGL.so.1: cannot open shared object file: No such file or directory
# !apt install -y libgl1-mesa-glx

In [1]:
import torch

print(torch.cuda.is_available())

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on ', device)
print(f"Device count: {torch.cuda.device_count()}")

True
Running on  cuda:0
Device count: 1


In [2]:
import os
import zipfile

CURRENT_PATH = f'{os.getcwd()}/'

#### Unzip nuScenes dataset

In [3]:
# DatasetName = 'Dataset.zip'
#
# print(f'{CURRENT_PATH}{DatasetName}')
#
# with zipfile.ZipFile(DatasetName, 'r') as zip_ref:
#     zip_ref.extractall(f'{CURRENT_PATH}{DatasetName}')

#### NuScenes initialization

In [3]:
from nuscenes import NuScenes
from nuscenes.prediction import PredictHelper
from nuscenes.eval.prediction.splits import get_prediction_challenge_split

import matplotlib.pyplot as plt

# This is the path where you stored your copy of the nuScenes dataset.
DATAROOT = 'Dataset/'

history_length = 2
prediction_length = 6

# Use v1.0-trainval or v1.0-mini
nusc = NuScenes('v1.0-trainval', dataroot=DATAROOT, verbose=False)
helper = PredictHelper(nusc)

In [4]:
train = get_prediction_challenge_split("train", dataroot=DATAROOT)
validation = get_prediction_challenge_split("train_val", dataroot=DATAROOT)
test = get_prediction_challenge_split("val", dataroot=DATAROOT)

print(f"Train len: {len(train)}\nVal len: {len(validation)}\nTest len: {len(test)}")

Train len: 32186
Val len: 8560
Test len: 9041


In [5]:
train = train[:12000]
validation = validation[:5000]
test = test[:5000]

In [6]:
import pickle

PATH_TO_EPSILON_8_SET = f"{DATAROOT}prediction_trajectory_sets/epsilon_8.pkl"
trajectories_set_8 = pickle.load(open(PATH_TO_EPSILON_8_SET, 'rb'))
trajectories_set_8 = torch.Tensor(trajectories_set_8)

#### Init datasets and dataloaders

In [7]:
from torch.utils.data import DataLoader, Dataset

from nuscenes.prediction.input_representation.static_layers import StaticLayerRasterizer

import numpy as np
from typing import List

class NuscenesDataset(Dataset):
    def __init__(self, tokens: List[str], helper: PredictHelper):
        self.tokens = tokens
        self.static_layer_representation = StaticLayerRasterizer(helper)

    def __len__(self):
        return len(self.tokens)
    
    def __getitem__(self, index: int):

        token = self.tokens[index]
        instance_token, sample_token = token.split("_")

        image = self.static_layer_representation.make_representation(instance_token, sample_token)
        image = torch.Tensor(image).permute(2, 0, 1)

        # NaN Values processing
        def agent_param_processing(value):
            if np.isnan(value):
                return -1
            return value
        
        vel = helper.get_velocity_for_agent(instance_token, sample_token)
        vel = agent_param_processing(vel)
        
        accel = helper.get_acceleration_for_agent(instance_token, sample_token)
        accel = agent_param_processing(accel)
        
        heading_cr = helper.get_heading_change_rate_for_agent(instance_token, sample_token)
        heading_cr = agent_param_processing(heading_cr)
                
        agent_state_vector = torch.Tensor([vel, accel, heading_cr])

        ground_truth = helper.get_future_for_agent(instance_token, sample_token, prediction_length, in_agent_frame=True)

        # Convert to [batch_size, 1, 12, 2]
        # Because loss function need that format
        ground_truth = np.expand_dims(ground_truth, 0)

        return image, agent_state_vector, ground_truth

In [8]:
batch_size = 32

train_ds = NuscenesDataset(train, helper)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

train_val_ds = NuscenesDataset(validation, helper)
train_val_dl = DataLoader(train_ds, batch_size=batch_size * 2)

In [9]:
image, state, ground_truth = next(iter(train_dl))
print(image.size())
print(state.size())
print(ground_truth.size())

print("Preprocessing states:")
print(state)

torch.Size([32, 3, 500, 500])
torch.Size([32, 3])
torch.Size([32, 1, 12, 2])
Preprocessing states:
tensor([[ 3.8829e+00,  4.2858e+00,  3.4914e-02],
        [ 7.4828e+00,  9.7190e-01,  0.0000e+00],
        [ 2.6254e+00,  1.1437e-01,  9.6029e-02],
        [ 5.5119e+00,  7.8166e-01,  3.1396e-01],
        [ 1.7754e+00,  2.1683e-01, -1.2660e-01],
        [ 4.9390e+00,  2.2711e-03, -3.4954e-02],
        [ 6.4788e-01, -2.9224e-04,  0.0000e+00],
        [ 1.3208e+00,  1.7265e+00, -4.1853e-02],
        [ 1.0706e+01, -2.5083e-03,  1.7458e-02],
        [ 1.2010e+01,  1.1310e+00,  8.8835e-16],
        [ 5.2919e+00, -1.8864e+00,  3.3168e-01],
        [ 9.6307e+00, -4.3436e-01,  0.0000e+00],
        [ 1.0469e+01, -1.0000e+00,  8.7288e-03],
        [ 5.1837e+00, -3.1336e-01, -5.0745e-01],
        [ 9.7165e-01, -2.1897e+00,  0.0000e+00],
        [ 8.9414e+00, -1.0000e+00,  0.0000e+00],
        [ 9.0079e+00, -5.9894e-03,  0.0000e+00],
        [ 1.9260e+00,  1.9186e+00, -1.0463e-02],
        [ 1.6071e+0

#### Init ML prediction model

In [10]:
from nuscenes.prediction.models.backbone import ResNetBackbone
import torchvision.models as models

# Torchvision backbone
backbone = models.resnext50_32x4d(pretrained=True)

# Build-in backbone
#backbone = ResNetBackbone('resnet50')

# Set backbone to non-trainable
def set_parameter_requires_grad(model):
    for param in model.parameters():
        param.requires_grad = False
        
set_parameter_requires_grad(backbone)

In [11]:
from torch.optim import SGD
from nuscenes.prediction.models.covernet import CoverNet, ConstantLatticeLoss

NUM_MODES = 64

model = CoverNet(backbone, num_modes=NUM_MODES)
model = model.to(device)

loss_function = ConstantLatticeLoss(trajectories_set_8)

c:\users\impor\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [12]:
# Pass to optimizer only params with requires_grad
params_to_update = []

for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

optimizer = SGD(params_to_update, lr=5e-4, momentum=0.9, weight_decay=5e-4)

	 head.0.weight
	 head.0.bias
	 head.1.weight
	 head.1.bias


In [13]:
from tqdm import tqdm
import copy
import time

def loss_batch(model, loss_func, img, state_vec, ground_truth, opt=None):
    img = img.to(device)
    state_vec = state_vec.to(device)
    ground_truth = ground_truth.to(device)
    
    predicted_logits = model(img, state_vec)
    loss = loss_func(predicted_logits, ground_truth)

    # For validation optimizer is None, thus we dont perform backprop
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    # Return losses and amount of items
    # print(f"{loss.item()}; {len(img)}")
    return loss.item(), len(img)


def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    best_loss = 999.0
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(epochs):
        start_epoch_time = time.time()
        print(f'Epoch: {epoch + 1}/{epochs}')
        print('-' * 10)
        
        model.train()

        for img, state_vec, gt in tqdm(train_dl):
            loss_batch(model, loss_func, img, state_vec, gt, opt)

        model.eval()
        print("Validation step")

        with torch.no_grad():
            # TODO: Using tqdm
            losses, nums = zip(
                *[loss_batch(model, loss_func, img, state_vec, gt) for img, state_vec, gt in valid_dl]
            )

        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        
        # deep copy the model
        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())

        print(f"Epoch {epoch + 1}; Loss: {val_loss:0.2f}; Best: {best_loss:0.2f} Time: {(time.time() - start_epoch_time):0.2f} sec;")


In [14]:
# epochs = 3  # how many epochs to train for
# fit(epochs, model, loss_function, optimizer, train_dl, train_val_dl)

In [15]:
# torch.save(model.state_dict(), '/root/model.pth')

In [16]:
model.load_state_dict(torch.load('./Models/model_data_20k_e25_loss_1-50.pth'))
model.eval()

CoverNet(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Seq

#### Metrics

In [17]:
NPY_DATA_PATH = './NpyDataset/'

test_features = np.load(f'{NPY_DATA_PATH}data_test_features_5k.npy')
test_states = np.load(f'{NPY_DATA_PATH}data_test_states_5k.npy')
test_labels = np.load(f'{NPY_DATA_PATH}data_test_labels_5k.npy')

In [18]:
def PlotPrediction(future, predict):
  plt.figure(figsize=(6, 6))

  plt.scatter(future[:, 1], -future[:, 0], c='orange', s=10)
  plt.scatter(predict[:, 1], -predict[:, 0], c='g', s=10)

  # Keep aspect ratio of axis
  plt.axis('equal')
  plt.show()


In [19]:
import nuscenes.eval.prediction.metrics as metrics

trajectories_set_8_np = trajectories_set_8.numpy()

metric_functions = [metrics.MinFDEK([1], aggregators=[metrics.RowMean()]),
                    metrics.MinADEK([5, 10], aggregators=[metrics.RowMean()]),
                    metrics.MissRateTopK([5, 10], tolerance=2, aggregators=[metrics.RowMean()])]

num_predictions = len(test_labels) # Amount of prediction rows

metrics_container = {metric.name: np.zeros((num_predictions, metric.shape)) for metric in metric_functions}
metrics_container

{'MinFDEK': array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]),
 'MinADEK': array([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]]),
 'MissRateTopK_2': array([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]])}

In [20]:
from tqdm import tqdm

for idx, x in enumerate(tqdm(test_labels)):
#for idx in range(len(test_labels)):
    # Make prediction
    img = torch.Tensor(test_features[idx].reshape((500, 500, 3))).permute(2, 0, 1).unsqueeze(0)
    img = img.to(device)

    state = torch.Tensor(np.array([test_states[idx]])).to(device)
    state = state.to(device)

    logits = model(img, state)
    mode_probabilities = np.array([logits.cpu().detach().numpy()[0]])[0]

    # Create prediction object
    instance_tkn, sample_tkn = test[idx].split("_")
    prediction = metrics.Prediction(instance_tkn, sample_tkn, trajectories_set_8_np, mode_probabilities)

    # Get ground_truth
    gt = test_labels[idx].reshape((12, 2))

    # Calculate metrics
    for metric in metric_functions:
        metrics_container[metric.name][idx] = metric(gt, prediction)

100%|██████████| 5000/5000 [03:35<00:00, 23.21it/s]


In [21]:
from collections import defaultdict
from typing import List, Dict, Any

aggregations: Dict[str, Dict[str, List[float]]] = defaultdict(dict)

for metric in metric_functions:
    for agg in metric.aggregators:
        aggregations[metric.name][agg.name] = agg(metrics_container[metric.name])

aggregations


defaultdict(dict,
            {'MinFDEK': {'RowMean': [12.143089777763683]},
             'MinADEK': {'RowMean': [2.5416744372181608, 2.20823303355608]},
             'MissRateTopK_2': {'RowMean': [0.9212, 0.9182]}})

In [42]:
categories = []

for sample in train:
    instance_tkn, sample_tkn =  sample.split("_")
    instance_category_token = nusc.get('instance', instance_tkn)['category_token']

    category_name = nusc.get('category', instance_category_token)['name']

    if category_name not in categories:
        categories.append(category_name)

In [35]:

categories

['vehicle.bus.rigid',
 'vehicle.car',
 'vehicle.truck',
 'vehicle.construction',
 'vehicle.bus.bendy',
 'vehicle.emergency.police']